In [1]:
import findspark
findspark.init('/usr/hdp/current/spark2-client')
findspark.find()

'/usr/hdp/current/spark2-client'

In [2]:
import pyspark

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("yarn").appName("ch06DifferentTypesOfData").getOrCreate()

In [4]:
sc = spark.sparkContext

In [5]:
spark

In [6]:
for x in sc._conf.getAll():
    if '/proxy/' in x[1]:
        print(x[1])

http://rm01.itversity.com:19288/proxy/application_1533622723243_14363


In [7]:
df = spark.read.format("csv")\
  .option("header", "true")\
  .option("inferSchema", "true")\
  .load("/user/kranthidr/dataSets/spark-guide/retail-data/by-day/2010-12-01.csv")

In [8]:
df.printSchema()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: timestamp (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: double (nullable = true)
 |-- Country: string (nullable = true)



In [9]:
df.show(5, False)

+---------+---------+-----------------------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|Description                        |Quantity|InvoiceDate        |UnitPrice|CustomerID|Country       |
+---------+---------+-----------------------------------+--------+-------------------+---------+----------+--------------+
|536365   |85123A   |WHITE HANGING HEART T-LIGHT HOLDER |6       |2010-12-01 08:26:00|2.55     |17850.0   |United Kingdom|
|536365   |71053    |WHITE METAL LANTERN                |6       |2010-12-01 08:26:00|3.39     |17850.0   |United Kingdom|
|536365   |84406B   |CREAM CUPID HEARTS COAT HANGER     |8       |2010-12-01 08:26:00|2.75     |17850.0   |United Kingdom|
|536365   |84029G   |KNITTED UNION FLAG HOT WATER BOTTLE|6       |2010-12-01 08:26:00|3.39     |17850.0   |United Kingdom|
|536365   |84029E   |RED WOOLLY HOTTIE WHITE HEART.     |6       |2010-12-01 08:26:00|3.39     |17850.0   |United Kingdom|
+---------+-----

In [10]:
df.createOrReplaceTempView("dfTable")

In [11]:
# COMMAND ----------

from pyspark.sql.functions import lit
lit_df = df.select(lit(5), lit("five"), lit(5.0))
lit_df.show(5)

+---+----+---+
|  5|five|5.0|
+---+----+---+
|  5|five|5.0|
|  5|five|5.0|
|  5|five|5.0|
|  5|five|5.0|
|  5|five|5.0|
+---+----+---+
only showing top 5 rows



In [12]:
lit_df.schema

StructType(List(StructField(5,IntegerType,false),StructField(five,StringType,false),StructField(5.0,DoubleType,false)))

In [13]:
# COMMAND ----------

from pyspark.sql.functions import col

df.where(col("InvoiceNo") != 536365)\
  .select("InvoiceNo", "Description")\
  .show(5, False)

+---------+-----------------------------+
|InvoiceNo|Description                  |
+---------+-----------------------------+
|536366   |HAND WARMER UNION JACK       |
|536366   |HAND WARMER RED POLKA DOT    |
|536367   |ASSORTED COLOUR BIRD ORNAMENT|
|536367   |POPPY'S PLAYHOUSE BEDROOM    |
|536367   |POPPY'S PLAYHOUSE KITCHEN    |
+---------+-----------------------------+
only showing top 5 rows



In [14]:
df.where("InvoiceNo != 536365")\
  .select("InvoiceNo", "Description")\
  .show(5, False)

+---------+-----------------------------+
|InvoiceNo|Description                  |
+---------+-----------------------------+
|536366   |HAND WARMER UNION JACK       |
|536366   |HAND WARMER RED POLKA DOT    |
|536367   |ASSORTED COLOUR BIRD ORNAMENT|
|536367   |POPPY'S PLAYHOUSE BEDROOM    |
|536367   |POPPY'S PLAYHOUSE KITCHEN    |
+---------+-----------------------------+
only showing top 5 rows



In [15]:
df.where("InvoiceNo <> 536365")\
  .select("InvoiceNo", "Description")\
  .show(5, False)

+---------+-----------------------------+
|InvoiceNo|Description                  |
+---------+-----------------------------+
|536366   |HAND WARMER UNION JACK       |
|536366   |HAND WARMER RED POLKA DOT    |
|536367   |ASSORTED COLOUR BIRD ORNAMENT|
|536367   |POPPY'S PLAYHOUSE BEDROOM    |
|536367   |POPPY'S PLAYHOUSE KITCHEN    |
+---------+-----------------------------+
only showing top 5 rows



In [16]:
#Both = and == will work......... for col == should be used
#even Invoice COlumn is String Type ...comparision with integer will work
df.where(col("InvoiceNo") == 536365)\
  .select("InvoiceNo", "Description")\
  .show(5, False)

+---------+-----------------------------------+
|InvoiceNo|Description                        |
+---------+-----------------------------------+
|536365   |WHITE HANGING HEART T-LIGHT HOLDER |
|536365   |WHITE METAL LANTERN                |
|536365   |CREAM CUPID HEARTS COAT HANGER     |
|536365   |KNITTED UNION FLAG HOT WATER BOTTLE|
|536365   |RED WOOLLY HOTTIE WHITE HEART.     |
+---------+-----------------------------------+
only showing top 5 rows



In [17]:
#Both = and == will work......... for col == should be used
df.where("InvoiceNo == 536365")\
  .select("InvoiceNo", "Description")\
  .show(5, False)

+---------+-----------------------------------+
|InvoiceNo|Description                        |
+---------+-----------------------------------+
|536365   |WHITE HANGING HEART T-LIGHT HOLDER |
|536365   |WHITE METAL LANTERN                |
|536365   |CREAM CUPID HEARTS COAT HANGER     |
|536365   |KNITTED UNION FLAG HOT WATER BOTTLE|
|536365   |RED WOOLLY HOTTIE WHITE HEART.     |
+---------+-----------------------------------+
only showing top 5 rows



In [18]:
# COMMAND ----------

from pyspark.sql.functions import instr
priceFilter = col("UnitPrice") > 600
descripFilter = instr(df.Description, "POSTAGE") >= 1

df.where(df.StockCode.isin("DOT")).where(priceFilter | descripFilter).show(5, False)

+---------+---------+--------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|Description   |Quantity|InvoiceDate        |UnitPrice|CustomerID|Country       |
+---------+---------+--------------+--------+-------------------+---------+----------+--------------+
|536544   |DOT      |DOTCOM POSTAGE|1       |2010-12-01 14:32:00|569.77   |null      |United Kingdom|
|536592   |DOT      |DOTCOM POSTAGE|1       |2010-12-01 17:06:00|607.49   |null      |United Kingdom|
+---------+---------+--------------+--------+-------------------+---------+----------+--------------+



In [19]:
df.where(df.Description.like("%DOT%")).show(5, False)

+---------+---------+---------------------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|Description                      |Quantity|InvoiceDate        |UnitPrice|CustomerID|Country       |
+---------+---------+---------------------------------+--------+-------------------+---------+----------+--------------+
|536366   |22632    |HAND WARMER RED POLKA DOT        |6       |2010-12-01 08:28:00|1.85     |17850.0   |United Kingdom|
|536372   |22632    |HAND WARMER RED POLKA DOT        |6       |2010-12-01 09:01:00|1.85     |17850.0   |United Kingdom|
|536377   |22632    |HAND WARMER RED POLKA DOT        |6       |2010-12-01 09:34:00|1.85     |17850.0   |United Kingdom|
|536378   |22386    |JUMBO BAG PINK POLKADOT          |10      |2010-12-01 09:37:00|1.95     |14688.0   |United Kingdom|
|536378   |84997C   |BLUE 3 PIECE POLKADOT CUTLERY SET|6       |2010-12-01 09:37:00|3.75     |14688.0   |United Kingdom|
+---------+---------+-----------

In [20]:
df.where(descripFilter).show(5, False)

+---------+---------+--------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|Description   |Quantity|InvoiceDate        |UnitPrice|CustomerID|Country       |
+---------+---------+--------------+--------+-------------------+---------+----------+--------------+
|536370   |POST     |POSTAGE       |3       |2010-12-01 08:45:00|18.0     |12583.0   |France        |
|536403   |POST     |POSTAGE       |1       |2010-12-01 11:27:00|15.0     |12791.0   |Netherlands   |
|536527   |POST     |POSTAGE       |1       |2010-12-01 13:04:00|18.0     |12662.0   |Germany       |
|536544   |DOT      |DOTCOM POSTAGE|1       |2010-12-01 14:32:00|569.77   |null      |United Kingdom|
|536592   |DOT      |DOTCOM POSTAGE|1       |2010-12-01 17:06:00|607.49   |null      |United Kingdom|
+---------+---------+--------------+--------+-------------------+---------+----------+--------------+



In [21]:
df.where(df.Description.isin("POSTAGE")).show(5, False)

+---------+---------+-----------+--------+-------------------+---------+----------+-----------+
|InvoiceNo|StockCode|Description|Quantity|InvoiceDate        |UnitPrice|CustomerID|Country    |
+---------+---------+-----------+--------+-------------------+---------+----------+-----------+
|536370   |POST     |POSTAGE    |3       |2010-12-01 08:45:00|18.0     |12583.0   |France     |
|536403   |POST     |POSTAGE    |1       |2010-12-01 11:27:00|15.0     |12791.0   |Netherlands|
|536527   |POST     |POSTAGE    |1       |2010-12-01 13:04:00|18.0     |12662.0   |Germany    |
+---------+---------+-----------+--------+-------------------+---------+----------+-----------+



In [22]:
df.where(priceFilter | descripFilter).show(5, False)

+---------+---------+--------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|Description   |Quantity|InvoiceDate        |UnitPrice|CustomerID|Country       |
+---------+---------+--------------+--------+-------------------+---------+----------+--------------+
|536370   |POST     |POSTAGE       |3       |2010-12-01 08:45:00|18.0     |12583.0   |France        |
|536403   |POST     |POSTAGE       |1       |2010-12-01 11:27:00|15.0     |12791.0   |Netherlands   |
|536527   |POST     |POSTAGE       |1       |2010-12-01 13:04:00|18.0     |12662.0   |Germany       |
|536544   |DOT      |DOTCOM POSTAGE|1       |2010-12-01 14:32:00|569.77   |null      |United Kingdom|
|536592   |DOT      |DOTCOM POSTAGE|1       |2010-12-01 17:06:00|607.49   |null      |United Kingdom|
+---------+---------+--------------+--------+-------------------+---------+----------+--------------+



In [23]:
df.where(df.StockCode.isin("DOT")).show(5, False)

+---------+---------+--------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|Description   |Quantity|InvoiceDate        |UnitPrice|CustomerID|Country       |
+---------+---------+--------------+--------+-------------------+---------+----------+--------------+
|536544   |DOT      |DOTCOM POSTAGE|1       |2010-12-01 14:32:00|569.77   |null      |United Kingdom|
|536592   |DOT      |DOTCOM POSTAGE|1       |2010-12-01 17:06:00|607.49   |null      |United Kingdom|
+---------+---------+--------------+--------+-------------------+---------+----------+--------------+



In [24]:
# COMMAND ----------

from pyspark.sql.functions import instr
DOTCodeFilter = col("StockCode") == "DOT"
priceFilter = col("UnitPrice") > 600
descripFilter = instr(col("Description"), "POSTAGE") >= 1
df.withColumn("isExpensive", DOTCodeFilter & (priceFilter | descripFilter))\
  .where("isExpensive")\
  .select("StockCode","Description","unitPrice", "isExpensive").show(5, False)

+---------+--------------+---------+-----------+
|StockCode|Description   |unitPrice|isExpensive|
+---------+--------------+---------+-----------+
|DOT      |DOTCOM POSTAGE|569.77   |true       |
|DOT      |DOTCOM POSTAGE|607.49   |true       |
+---------+--------------+---------+-----------+



In [25]:
df.withColumn("isExpensive", DOTCodeFilter & (priceFilter | descripFilter))\
  .select("StockCode","Description","unitPrice", "isExpensive").show(5, False)

+---------+-----------------------------------+---------+-----------+
|StockCode|Description                        |unitPrice|isExpensive|
+---------+-----------------------------------+---------+-----------+
|85123A   |WHITE HANGING HEART T-LIGHT HOLDER |2.55     |false      |
|71053    |WHITE METAL LANTERN                |3.39     |false      |
|84406B   |CREAM CUPID HEARTS COAT HANGER     |2.75     |false      |
|84029G   |KNITTED UNION FLAG HOT WATER BOTTLE|3.39     |false      |
|84029E   |RED WOOLLY HOTTIE WHITE HEART.     |3.39     |false      |
+---------+-----------------------------------+---------+-----------+
only showing top 5 rows



In [26]:
# COMMAND ----------

from pyspark.sql.functions import expr
df.withColumn("isExpensive", expr("NOT UnitPrice <= 250"))\
  .where("isExpensive")\
  .select("StockCode","Description","unitPrice", "isExpensive").show(5, False)

+---------+--------------+---------+-----------+
|StockCode|Description   |unitPrice|isExpensive|
+---------+--------------+---------+-----------+
|DOT      |DOTCOM POSTAGE|569.77   |true       |
|DOT      |DOTCOM POSTAGE|607.49   |true       |
+---------+--------------+---------+-----------+



In [27]:
df.withColumn("isExpensive", expr("UnitPrice > 250"))\
  .where("isExpensive")\
  .select("StockCode","Description","unitPrice", "isExpensive").show(5, False)

+---------+--------------+---------+-----------+
|StockCode|Description   |unitPrice|isExpensive|
+---------+--------------+---------+-----------+
|DOT      |DOTCOM POSTAGE|569.77   |true       |
|DOT      |DOTCOM POSTAGE|607.49   |true       |
+---------+--------------+---------+-----------+



In [28]:
# COMMAND ----------

from pyspark.sql.functions import expr, pow

fabricatedQuantity = pow(col("Quantity") * col("UnitPrice"), 2) + 5
df.select(expr("CustomerId"), fabricatedQuantity.alias("realQuantity")).show(2)

+----------+------------------+
|CustomerId|      realQuantity|
+----------+------------------+
|   17850.0|239.08999999999997|
|   17850.0|          418.7156|
+----------+------------------+
only showing top 2 rows



In [29]:
# COMMAND ----------

df.selectExpr(
  "CustomerId",
  "(POWER((Quantity * UnitPrice), 2.0) + 5) as realQuantity").show(2)

+----------+------------------+
|CustomerId|      realQuantity|
+----------+------------------+
|   17850.0|239.08999999999997|
|   17850.0|          418.7156|
+----------+------------------+
only showing top 2 rows



In [30]:
# COMMAND ----------

from pyspark.sql.functions import lit, round, bround

df.select(round(lit("2.5")), bround(lit("2.5"))).show(2)

+-------------+--------------+
|round(2.5, 0)|bround(2.5, 0)|
+-------------+--------------+
|          3.0|           2.0|
|          3.0|           2.0|
+-------------+--------------+
only showing top 2 rows



In [31]:
# COMMAND ----------

from pyspark.sql.functions import corr
df.stat.corr("Quantity", "UnitPrice")

-0.04112314436835551

In [32]:
df.select(corr("Quantity", "UnitPrice")).show()

+-------------------------+
|corr(Quantity, UnitPrice)|
+-------------------------+
|     -0.04112314436835551|
+-------------------------+



In [33]:
# COMMAND ----------

df.describe().printSchema()

root
 |-- summary: string (nullable = true)
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: string (nullable = true)
 |-- UnitPrice: string (nullable = true)
 |-- CustomerID: string (nullable = true)
 |-- Country: string (nullable = true)



In [34]:
df.describe().columns

['summary',
 'InvoiceNo',
 'StockCode',
 'Description',
 'Quantity',
 'UnitPrice',
 'CustomerID',
 'Country']

In [35]:
df.describe().select('summary',
 'InvoiceNo',
 'StockCode',
 'Description',
 'Quantity').show()

+-------+-----------------+------------------+--------------------+------------------+
|summary|        InvoiceNo|         StockCode|         Description|          Quantity|
+-------+-----------------+------------------+--------------------+------------------+
|  count|             3108|              3108|                3098|              3108|
|   mean| 536516.684944841|27834.304044117645|                null| 8.627413127413128|
| stddev|72.89447869788873|17407.897548583845|                null|26.371821677029203|
|    min|           536365|             10002| 4 PURPLE FLOCK D...|               -24|
|    max|          C536548|              POST|ZINC WILLIE WINKI...|               600|
+-------+-----------------+------------------+--------------------+------------------+



In [36]:
df.describe().select('summary',
 'UnitPrice',
 'CustomerID',
 'Country').show()

+-------+------------------+------------------+--------------+
|summary|         UnitPrice|        CustomerID|       Country|
+-------+------------------+------------------+--------------+
|  count|              3108|              1968|          3108|
|   mean| 4.151946589446603|15661.388719512195|          null|
| stddev|15.638659854603892|1854.4496996893627|          null|
|    min|               0.0|           12431.0|     Australia|
|    max|            607.49|           18229.0|United Kingdom|
+-------+------------------+------------------+--------------+



In [37]:
# COMMAND ----------

from pyspark.sql.functions import count, mean, stddev_pop, min, max

In [38]:
# COMMAND ----------

colName = "UnitPrice"
quantileProbs = [0.5]
relError = 0.05
df.stat.approxQuantile("UnitPrice", quantileProbs, relError) # 2.51

[2.51]

In [39]:
df.stat.crosstab("StockCode", "Quantity").printSchema()

root
 |-- StockCode_Quantity: string (nullable = true)
 |-- -1: long (nullable = true)
 |-- -10: long (nullable = true)
 |-- -12: long (nullable = true)
 |-- -2: long (nullable = true)
 |-- -24: long (nullable = true)
 |-- -3: long (nullable = true)
 |-- -4: long (nullable = true)
 |-- -5: long (nullable = true)
 |-- -6: long (nullable = true)
 |-- -7: long (nullable = true)
 |-- 1: long (nullable = true)
 |-- 10: long (nullable = true)
 |-- 100: long (nullable = true)
 |-- 11: long (nullable = true)
 |-- 12: long (nullable = true)
 |-- 120: long (nullable = true)
 |-- 128: long (nullable = true)
 |-- 13: long (nullable = true)
 |-- 14: long (nullable = true)
 |-- 144: long (nullable = true)
 |-- 15: long (nullable = true)
 |-- 16: long (nullable = true)
 |-- 17: long (nullable = true)
 |-- 18: long (nullable = true)
 |-- 19: long (nullable = true)
 |-- 192: long (nullable = true)
 |-- 2: long (nullable = true)
 |-- 20: long (nullable = true)
 |-- 200: long (nullable = true)
 |-- 21: l

In [40]:
# COMMAND ----------

df.stat.crosstab("StockCode", "Quantity").take(3)

[Row(StockCode_Quantity=u'22578', -1=0, -10=0, -12=0, -2=0, -24=0, -3=0, -4=0, -5=0, -6=0, -7=0, 1=0, 10=0, 100=0, 11=0, 12=0, 120=0, 128=0, 13=0, 14=0, 144=0, 15=0, 16=0, 17=0, 18=0, 19=0, 192=0, 2=0, 20=0, 200=0, 21=0, 216=0, 22=0, 23=0, 24=1, 25=0, 252=0, 27=0, 28=0, 288=0, 3=0, 30=0, 32=0, 33=0, 34=0, 36=0, 384=0, 4=0, 40=0, 432=0, 47=0, 48=0, 480=0, 5=0, 50=0, 56=0, 6=0, 60=0, 600=0, 64=0, 7=0, 70=0, 72=0, 8=0, 80=0, 9=0, 96=0),
 Row(StockCode_Quantity=u'21327', -1=0, -10=0, -12=0, -2=0, -24=0, -3=0, -4=0, -5=0, -6=0, -7=0, 1=2, 10=0, 100=0, 11=0, 12=0, 120=0, 128=0, 13=0, 14=0, 144=0, 15=0, 16=0, 17=0, 18=0, 19=0, 192=0, 2=0, 20=0, 200=0, 21=0, 216=0, 22=0, 23=0, 24=0, 25=0, 252=0, 27=0, 28=0, 288=0, 3=0, 30=0, 32=0, 33=0, 34=0, 36=0, 384=0, 4=0, 40=0, 432=0, 47=0, 48=0, 480=0, 5=0, 50=0, 56=0, 6=0, 60=0, 600=0, 64=0, 7=0, 70=0, 72=0, 8=0, 80=0, 9=0, 96=0),
 Row(StockCode_Quantity=u'22064', -1=0, -10=0, -12=0, -2=0, -24=0, -3=0, -4=0, -5=0, -6=0, -7=0, 1=1, 10=0, 100=0, 11=0, 12=

In [41]:
# COMMAND ----------
df.stat.freqItems(["StockCode", "Quantity"]).printSchema()
df.stat.freqItems(["StockCode", "Quantity"]).take(5)

root
 |-- StockCode_freqItems: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- Quantity_freqItems: array (nullable = true)
 |    |-- element: integer (containsNull = false)



[Row(StockCode_freqItems=[u'90214E', u'20728', u'20755', u'21703', u'22113', u'22524', u'22041', u'72803A', u'72798C', u'90181B', u'21756', u'22694', u'90206C', u'20970', u'21624', u'90209C', u'84744', u'82494L', u'22952', u'20682', u'22583', u'21705', u'20679', u'22220', u'90177E', u'90214A', u'22448', u'90214S', u'22121', u'22802', u'84970L', u'72818', u'90192', u'90200C', u'22910', u'21380', u'90211A', u'21137', u'35271S', u'84926A', u'20765', u'22384', u'21524', u'22165', u'22366', u'21221', u'21704', u'22519', u'85035C', u'21967', u'22114', u'22909', u'22900', u'22447', u'21577', u'21877', u'20726', u'85034A', u'DOT', u'84658', u'21472', u'22804', u'22222', u'72802C', u'21739', u'22467', u'90214H', u'22785', u'22446', u'22197', u'20665', u'21733', u'22731', u'21709', u'22086', u'40001', u'85123A'], Quantity_freqItems=[200, 128, 23, 32, 50, 600, 8, 17, 80, -1, -10, 11, 56, 47, 20, -7, 2, 5, 480, -4, 14, 432, 100, 64, 40, 13, 4, -5, 22, 16, -2, 7, 70, 384, 25, 34, 10, 1, 288, 216, 2

In [42]:
# COMMAND ----------

from pyspark.sql.functions import monotonically_increasing_id
df = df.withColumn("index", monotonically_increasing_id())

In [43]:
df.show(5,False)

+---------+---------+-----------------------------------+--------+-------------------+---------+----------+--------------+-----+
|InvoiceNo|StockCode|Description                        |Quantity|InvoiceDate        |UnitPrice|CustomerID|Country       |index|
+---------+---------+-----------------------------------+--------+-------------------+---------+----------+--------------+-----+
|536365   |85123A   |WHITE HANGING HEART T-LIGHT HOLDER |6       |2010-12-01 08:26:00|2.55     |17850.0   |United Kingdom|0    |
|536365   |71053    |WHITE METAL LANTERN                |6       |2010-12-01 08:26:00|3.39     |17850.0   |United Kingdom|1    |
|536365   |84406B   |CREAM CUPID HEARTS COAT HANGER     |8       |2010-12-01 08:26:00|2.75     |17850.0   |United Kingdom|2    |
|536365   |84029G   |KNITTED UNION FLAG HOT WATER BOTTLE|6       |2010-12-01 08:26:00|3.39     |17850.0   |United Kingdom|3    |
|536365   |84029E   |RED WOOLLY HOTTIE WHITE HEART.     |6       |2010-12-01 08:26:00|3.39     |1

In [44]:
df.count()

3108

In [45]:
df.filter("index > 3095").show(30,False)

+---------+---------+-----------------------------------+--------+-------------------+---------+----------+--------------+-----+
|InvoiceNo|StockCode|Description                        |Quantity|InvoiceDate        |UnitPrice|CustomerID|Country       |index|
+---------+---------+-----------------------------------+--------+-------------------+---------+----------+--------------+-----+
|536597   |21739    |COSY SLIPPER SHOES SMALL GREEN     |1       |2010-12-01 17:35:00|2.95     |18011.0   |United Kingdom|3096 |
|536597   |85034A   |3 GARDENIA MORRIS BOXED CANDLES    |1       |2010-12-01 17:35:00|4.25     |18011.0   |United Kingdom|3097 |
|536597   |72818    |CHRISTMAS DECOUPAGE CANDLE         |4       |2010-12-01 17:35:00|0.85     |18011.0   |United Kingdom|3098 |
|536597   |72798C   |SET/4 GARDEN ROSE DINNER CANDLE    |1       |2010-12-01 17:35:00|1.65     |18011.0   |United Kingdom|3099 |
|536597   |22197    |SMALL POPCORN HOLDER               |6       |2010-12-01 17:35:00|0.85     |1

In [46]:
# COMMAND ----------

from pyspark.sql.functions import initcap
df.select(initcap(col("Description"))).show(4,False)

+-----------------------------------+
|initcap(Description)               |
+-----------------------------------+
|White Hanging Heart T-light Holder |
|White Metal Lantern                |
|Cream Cupid Hearts Coat Hanger     |
|Knitted Union Flag Hot Water Bottle|
+-----------------------------------+
only showing top 4 rows



In [47]:
# COMMAND ----------

from pyspark.sql.functions import lower, upper
df.select(col("Description"),
    lower(col("Description")),
    upper(lower(col("Description")))).show(3, False)

+----------------------------------+----------------------------------+----------------------------------+
|Description                       |lower(Description)                |upper(lower(Description))         |
+----------------------------------+----------------------------------+----------------------------------+
|WHITE HANGING HEART T-LIGHT HOLDER|white hanging heart t-light holder|WHITE HANGING HEART T-LIGHT HOLDER|
|WHITE METAL LANTERN               |white metal lantern               |WHITE METAL LANTERN               |
|CREAM CUPID HEARTS COAT HANGER    |cream cupid hearts coat hanger    |CREAM CUPID HEARTS COAT HANGER    |
+----------------------------------+----------------------------------+----------------------------------+
only showing top 3 rows



In [48]:
# COMMAND ----------

from pyspark.sql.functions import lit, ltrim, rtrim, rpad, lpad, trim
df.select(
    ltrim(lit("    HELLO    ")).alias("ltrim"),
    rtrim(lit("    HELLO    ")).alias("rtrim"),
    trim(lit("    HELLO    ")).alias("trim"),
    lpad(lit("HELLO"), 3, " ").alias("lp3"),
    lpad(lit("HELLO"), 10, " ").alias("lp10"),
    rpad(lit("HELLO"), 3, " ").alias("rp3"),
    rpad(lit("HELLO"), 10, " ").alias("rp10")
).show(4, False)

+---------+---------+-----+---+----------+---+----------+
|ltrim    |rtrim    |trim |lp3|lp10      |rp3|rp10      |
+---------+---------+-----+---+----------+---+----------+
|HELLO    |    HELLO|HELLO|HEL|     HELLO|HEL|HELLO     |
|HELLO    |    HELLO|HELLO|HEL|     HELLO|HEL|HELLO     |
|HELLO    |    HELLO|HELLO|HEL|     HELLO|HEL|HELLO     |
|HELLO    |    HELLO|HELLO|HEL|     HELLO|HEL|HELLO     |
+---------+---------+-----+---+----------+---+----------+
only showing top 4 rows



In [49]:
# COMMAND ----------

from pyspark.sql.functions import regexp_replace

regex_string = "BLACK|WHITE|RED|GREEN|BLUE"
df.select(regexp_replace(col("Description"), regex_string, "COLOR").alias("color_clean"),
  col("Description")).show(2)

+--------------------+--------------------+
|         color_clean|         Description|
+--------------------+--------------------+
|COLOR HANGING HEA...|WHITE HANGING HEA...|
| COLOR METAL LANTERN| WHITE METAL LANTERN|
+--------------------+--------------------+
only showing top 2 rows



In [50]:
# COMMAND ----------

from pyspark.sql.functions import translate
df.select(translate(col("Description"), "LEET", "1337"),col("Description"))\
  .show(2)

+----------------------------------+--------------------+
|translate(Description, LEET, 1337)|         Description|
+----------------------------------+--------------------+
|              WHI73 HANGING H3A...|WHITE HANGING HEA...|
|               WHI73 M37A1 1AN73RN| WHITE METAL LANTERN|
+----------------------------------+--------------------+
only showing top 2 rows



In [51]:
# COMMAND ----------

from pyspark.sql.functions import regexp_extract
extract_str = "(BLACK|WHITE|RED|GREEN|BLUE)"
df.select(
     regexp_extract(col("Description"), extract_str, 1).alias("color_extracted"),
     col("Description")).show(2)

+---------------+--------------------+
|color_extracted|         Description|
+---------------+--------------------+
|          WHITE|WHITE HANGING HEA...|
|          WHITE| WHITE METAL LANTERN|
+---------------+--------------------+
only showing top 2 rows



In [52]:
# COMMAND ----------

from pyspark.sql.functions import instr
containsBlack = instr(col("Description"), "BLACK") >= 1
containsWhite = instr(col("Description"), "WHITE") >= 1
df.withColumn("hasSimpleColor", containsBlack | containsWhite)\
  .where("hasSimpleColor")\
  .select("Description").show(3, False)

+----------------------------------+
|Description                       |
+----------------------------------+
|WHITE HANGING HEART T-LIGHT HOLDER|
|WHITE METAL LANTERN               |
|RED WOOLLY HOTTIE WHITE HEART.    |
+----------------------------------+
only showing top 3 rows



In [53]:
# COMMAND ----------

from pyspark.sql.functions import expr, locate
simpleColors = ["black", "white", "red", "green", "blue"]

In [54]:
def color_locator(column, color_string):
  return locate(color_string.upper(), column)\
          .cast("boolean")\
          .alias("is_" + color_string)
        

In [55]:
selectedColumns1 = [color_locator(df.Description, c) for c in simpleColors]

In [56]:
selectedColumns1

[Column<CAST(locate(BLACK, Description, 1) AS BOOLEAN) AS `is_black`>,
 Column<CAST(locate(WHITE, Description, 1) AS BOOLEAN) AS `is_white`>,
 Column<CAST(locate(RED, Description, 1) AS BOOLEAN) AS `is_red`>,
 Column<CAST(locate(GREEN, Description, 1) AS BOOLEAN) AS `is_green`>,
 Column<CAST(locate(BLUE, Description, 1) AS BOOLEAN) AS `is_blue`>]

In [57]:
selectedColumns1.append(expr("*")) # has to a be Column type

In [58]:
selectedColumns1

[Column<CAST(locate(BLACK, Description, 1) AS BOOLEAN) AS `is_black`>,
 Column<CAST(locate(WHITE, Description, 1) AS BOOLEAN) AS `is_white`>,
 Column<CAST(locate(RED, Description, 1) AS BOOLEAN) AS `is_red`>,
 Column<CAST(locate(GREEN, Description, 1) AS BOOLEAN) AS `is_green`>,
 Column<CAST(locate(BLUE, Description, 1) AS BOOLEAN) AS `is_blue`>,
 Column<unresolvedstar()>]

In [59]:
df.select(*selectedColumns1).where(expr("is_white OR is_red"))\
  .select("is_black","is_white","is_red","is_green","is_blue","Description").show(30, False)

+--------+--------+------+--------+-------+----------------------------------+
|is_black|is_white|is_red|is_green|is_blue|Description                       |
+--------+--------+------+--------+-------+----------------------------------+
|false   |true    |false |false   |false  |WHITE HANGING HEART T-LIGHT HOLDER|
|false   |true    |false |false   |false  |WHITE METAL LANTERN               |
|false   |true    |true  |false   |false  |RED WOOLLY HOTTIE WHITE HEART.    |
|false   |false   |true  |false   |false  |HAND WARMER RED POLKA DOT         |
|false   |false   |true  |false   |false  |RED COAT RACK PARIS FASHION       |
|false   |false   |true  |false   |false  |ALARM CLOCK BAKELIKE RED          |
|false   |false   |true  |false   |false  |SET/2 RED RETROSPOT TEA TOWELS    |
|false   |false   |true  |false   |false  |RED TOADSTOOL LED NIGHT LIGHT     |
|false   |false   |true  |false   |false  |HAND WARMER RED POLKA DOT         |
|false   |true    |false |false   |false  |WHITE HAN

In [60]:
selectedColumns2 = [color_locator(df.Description, c) for c in simpleColors]
selectedColumns2.append(expr("*"))
#this is must for Var Args syntax
df.select(*selectedColumns2).where(expr("is_white OR is_red"))\
  .select("is_black","is_white","is_red","is_green","is_blue","Description").show(30, False)

+--------+--------+------+--------+-------+----------------------------------+
|is_black|is_white|is_red|is_green|is_blue|Description                       |
+--------+--------+------+--------+-------+----------------------------------+
|false   |true    |false |false   |false  |WHITE HANGING HEART T-LIGHT HOLDER|
|false   |true    |false |false   |false  |WHITE METAL LANTERN               |
|false   |true    |true  |false   |false  |RED WOOLLY HOTTIE WHITE HEART.    |
|false   |false   |true  |false   |false  |HAND WARMER RED POLKA DOT         |
|false   |false   |true  |false   |false  |RED COAT RACK PARIS FASHION       |
|false   |false   |true  |false   |false  |ALARM CLOCK BAKELIKE RED          |
|false   |false   |true  |false   |false  |SET/2 RED RETROSPOT TEA TOWELS    |
|false   |false   |true  |false   |false  |RED TOADSTOOL LED NIGHT LIGHT     |
|false   |false   |true  |false   |false  |HAND WARMER RED POLKA DOT         |
|false   |true    |false |false   |false  |WHITE HAN

In [61]:
# COMMAND ----------

from pyspark.sql.functions import current_date, current_timestamp
dateDF = spark.range(10)\
  .withColumn("today", current_date())\
  .withColumn("now", current_timestamp())
dateDF.createOrReplaceTempView("dateTable")

In [62]:
dateDF.show()

+---+----------+--------------------+
| id|     today|                 now|
+---+----------+--------------------+
|  0|2018-09-04|2018-09-04 04:56:...|
|  1|2018-09-04|2018-09-04 04:56:...|
|  2|2018-09-04|2018-09-04 04:56:...|
|  3|2018-09-04|2018-09-04 04:56:...|
|  4|2018-09-04|2018-09-04 04:56:...|
|  5|2018-09-04|2018-09-04 04:56:...|
|  6|2018-09-04|2018-09-04 04:56:...|
|  7|2018-09-04|2018-09-04 04:56:...|
|  8|2018-09-04|2018-09-04 04:56:...|
|  9|2018-09-04|2018-09-04 04:56:...|
+---+----------+--------------------+



In [63]:
# COMMAND ----------

from pyspark.sql.functions import date_add, date_sub
dateDF.select("*",date_sub(col("today"), 5), date_add(col("today"), 5)).show(6, False)

+---+----------+-----------------------+------------------+------------------+
|id |today     |now                    |date_sub(today, 5)|date_add(today, 5)|
+---+----------+-----------------------+------------------+------------------+
|0  |2018-09-04|2018-09-04 04:56:44.746|2018-08-30        |2018-09-09        |
|1  |2018-09-04|2018-09-04 04:56:44.746|2018-08-30        |2018-09-09        |
|2  |2018-09-04|2018-09-04 04:56:44.746|2018-08-30        |2018-09-09        |
|3  |2018-09-04|2018-09-04 04:56:44.746|2018-08-30        |2018-09-09        |
|4  |2018-09-04|2018-09-04 04:56:44.746|2018-08-30        |2018-09-09        |
|5  |2018-09-04|2018-09-04 04:56:44.746|2018-08-30        |2018-09-09        |
+---+----------+-----------------------+------------------+------------------+
only showing top 6 rows



In [64]:
# COMMAND ----------

from pyspark.sql.functions import datediff, months_between, to_date
dateDF.withColumn("week_ago", date_sub(col("today"), 7))\
  .select("*", datediff(col("week_ago"), col("today"))).show(5, False)

+---+----------+-----------------------+----------+-------------------------+
|id |today     |now                    |week_ago  |datediff(week_ago, today)|
+---+----------+-----------------------+----------+-------------------------+
|0  |2018-09-04|2018-09-04 04:56:44.894|2018-08-28|-7                       |
|1  |2018-09-04|2018-09-04 04:56:44.894|2018-08-28|-7                       |
|2  |2018-09-04|2018-09-04 04:56:44.894|2018-08-28|-7                       |
|3  |2018-09-04|2018-09-04 04:56:44.894|2018-08-28|-7                       |
|4  |2018-09-04|2018-09-04 04:56:44.894|2018-08-28|-7                       |
+---+----------+-----------------------+----------+-------------------------+
only showing top 5 rows



In [65]:
dateDF.select(
    to_date(lit("2016-01-01")).alias("start"),
    to_date(lit("2017-05-22")).alias("end"))\
  .select("*", months_between(col("start"), col("end"))).show(5, False)

+----------+----------+--------------------------+
|start     |end       |months_between(start, end)|
+----------+----------+--------------------------+
|2016-01-01|2017-05-22|-16.67741935              |
|2016-01-01|2017-05-22|-16.67741935              |
|2016-01-01|2017-05-22|-16.67741935              |
|2016-01-01|2017-05-22|-16.67741935              |
|2016-01-01|2017-05-22|-16.67741935              |
+----------+----------+--------------------------+
only showing top 5 rows



In [66]:
# COMMAND ----------

from pyspark.sql.functions import to_date, lit
dateDF = spark.range(5).withColumn("date", lit("2017-01-01"))\
  .select("*", to_date(col("date")))
dateDF.show(5,False)

+---+----------+---------------+
|id |date      |to_date(`date`)|
+---+----------+---------------+
|0  |2017-01-01|2017-01-01     |
|1  |2017-01-01|2017-01-01     |
|2  |2017-01-01|2017-01-01     |
|3  |2017-01-01|2017-01-01     |
|4  |2017-01-01|2017-01-01     |
+---+----------+---------------+



In [67]:
dateDF.printSchema()

root
 |-- id: long (nullable = false)
 |-- date: string (nullable = false)
 |-- to_date(`date`): date (nullable = true)



In [68]:
# COMMAND ----------

from pyspark.sql.functions import to_date
dateFormat = "yyyy-dd-MM"
cleanDateDF = spark.range(4).select(
    to_date(lit("2017-12-11"), dateFormat).alias("date"),
    to_date(lit("2017-20-12"), dateFormat).alias("date2"))
cleanDateDF.createOrReplaceTempView("dateTable2")

In [69]:
cleanDateDF.show(5, False)

+----------+----------+
|date      |date2     |
+----------+----------+
|2017-11-12|2017-12-20|
|2017-11-12|2017-12-20|
|2017-11-12|2017-12-20|
|2017-11-12|2017-12-20|
+----------+----------+



In [70]:
# COMMAND ----------

from pyspark.sql.functions import to_timestamp
cleanDateDF.select("*", to_timestamp(col("date"), dateFormat)).show(5, False)

+----------+----------+----------------------------------+
|date      |date2     |to_timestamp(`date`, 'yyyy-dd-MM')|
+----------+----------+----------------------------------+
|2017-11-12|2017-12-20|2017-11-12 00:00:00               |
|2017-11-12|2017-12-20|2017-11-12 00:00:00               |
|2017-11-12|2017-12-20|2017-11-12 00:00:00               |
|2017-11-12|2017-12-20|2017-11-12 00:00:00               |
+----------+----------+----------------------------------+



In [71]:
# cleanDateDF.select("*", to_timestamp(time_add(col("date"),5000), 'yyyy-MM-dd:mm-ss')).show(5, False)
#How to seconds / minutes/ hours

In [72]:
df.where(col("Description").eqNullSafe("WHITE METAL LANTERN")).show(5, False)

+---------+---------+-------------------+--------+-------------------+---------+----------+--------------+-----+
|InvoiceNo|StockCode|Description        |Quantity|InvoiceDate        |UnitPrice|CustomerID|Country       |index|
+---------+---------+-------------------+--------+-------------------+---------+----------+--------------+-----+
|536365   |71053    |WHITE METAL LANTERN|6       |2010-12-01 08:26:00|3.39     |17850.0   |United Kingdom|1    |
|536373   |71053    |WHITE METAL LANTERN|6       |2010-12-01 09:02:00|3.39     |17850.0   |United Kingdom|50   |
|536375   |71053    |WHITE METAL LANTERN|6       |2010-12-01 09:32:00|3.39     |17850.0   |United Kingdom|67   |
|536396   |71053    |WHITE METAL LANTERN|6       |2010-12-01 10:51:00|3.39     |17850.0   |United Kingdom|279  |
|536406   |71053    |WHITE METAL LANTERN|8       |2010-12-01 11:33:00|3.39     |17850.0   |United Kingdom|417  |
+---------+---------+-------------------+--------+-------------------+---------+----------+-----

In [73]:
df.where(col("Description") != "WHITE METAL LANTERN").where(col("Description").isNotNull()).show(5, False)

+---------+---------+-----------------------------------+--------+-------------------+---------+----------+--------------+-----+
|InvoiceNo|StockCode|Description                        |Quantity|InvoiceDate        |UnitPrice|CustomerID|Country       |index|
+---------+---------+-----------------------------------+--------+-------------------+---------+----------+--------------+-----+
|536365   |85123A   |WHITE HANGING HEART T-LIGHT HOLDER |6       |2010-12-01 08:26:00|2.55     |17850.0   |United Kingdom|0    |
|536365   |84406B   |CREAM CUPID HEARTS COAT HANGER     |8       |2010-12-01 08:26:00|2.75     |17850.0   |United Kingdom|2    |
|536365   |84029G   |KNITTED UNION FLAG HOT WATER BOTTLE|6       |2010-12-01 08:26:00|3.39     |17850.0   |United Kingdom|3    |
|536365   |84029E   |RED WOOLLY HOTTIE WHITE HEART.     |6       |2010-12-01 08:26:00|3.39     |17850.0   |United Kingdom|4    |
|536365   |22752    |SET 7 BABUSHKA NESTING BOXES       |2       |2010-12-01 08:26:00|7.65     |1

In [74]:
df.where(col("Description") != "WHITE METAL LANTERN").where(col("Description").isNull()).show(5, False)
#wher ignores Null values

+---------+---------+-----------+--------+-----------+---------+----------+-------+-----+
|InvoiceNo|StockCode|Description|Quantity|InvoiceDate|UnitPrice|CustomerID|Country|index|
+---------+---------+-----------+--------+-----------+---------+----------+-------+-----+
+---------+---------+-----------+--------+-----------+---------+----------+-------+-----+



In [75]:
df.where(col("Description").isNull()).show(5, False)

+---------+---------+-----------+--------+-------------------+---------+----------+--------------+-----+
|InvoiceNo|StockCode|Description|Quantity|InvoiceDate        |UnitPrice|CustomerID|Country       |index|
+---------+---------+-----------+--------+-------------------+---------+----------+--------------+-----+
|536414   |22139    |null       |56      |2010-12-01 11:52:00|0.0      |null      |United Kingdom|622  |
|536545   |21134    |null       |1       |2010-12-01 14:32:00|0.0      |null      |United Kingdom|1970 |
|536546   |22145    |null       |1       |2010-12-01 14:33:00|0.0      |null      |United Kingdom|1971 |
|536547   |37509    |null       |1       |2010-12-01 14:33:00|0.0      |null      |United Kingdom|1972 |
|536549   |85226A   |null       |1       |2010-12-01 14:34:00|0.0      |null      |United Kingdom|1987 |
+---------+---------+-----------+--------+-------------------+---------+----------+--------------+-----+
only showing top 5 rows



In [76]:
from pyspark.sql.functions import isnan, when, count, col

#df.select([count(when(isnan(c), c)).alias(c) for c in df.columns]).show() # not work due to Date Column
df.select([count(when(col(c).isNull() |isnan(c), c)).alias("count_"+c) \
           for c in [x for x in df.columns if x != "InvoiceDate"]]).show()

+---------------+---------------+-----------------+--------------+---------------+----------------+-------------+-----------+
|count_InvoiceNo|count_StockCode|count_Description|count_Quantity|count_UnitPrice|count_CustomerID|count_Country|count_index|
+---------------+---------------+-----------------+--------------+---------------+----------------+-------------+-----------+
|              0|              0|               10|             0|              0|            1140|            0|          0|
+---------------+---------------+-----------------+--------------+---------------+----------------+-------------+-----------+



In [77]:
df.select([count(when(isnan(c), c)).alias("count_"+c)\
           for c in [x for x in df.columns if x != "InvoiceDate"]]).show()

+---------------+---------------+-----------------+--------------+---------------+----------------+-------------+-----------+
|count_InvoiceNo|count_StockCode|count_Description|count_Quantity|count_UnitPrice|count_CustomerID|count_Country|count_index|
+---------------+---------------+-----------------+--------------+---------------+----------------+-------------+-----------+
|              0|              0|                0|             0|              0|               0|            0|          0|
+---------------+---------------+-----------------+--------------+---------------+----------------+-------------+-----------+



In [78]:
df.select([count(when(col(c).isNull(), c)).alias("cn_"+c)\
           for c in df.columns]).show(5,False)

+------------+------------+--------------+-----------+--------------+------------+-------------+----------+--------+
|cn_InvoiceNo|cn_StockCode|cn_Description|cn_Quantity|cn_InvoiceDate|cn_UnitPrice|cn_CustomerID|cn_Country|cn_index|
+------------+------------+--------------+-----------+--------------+------------+-------------+----------+--------+
|0           |0           |10            |0          |0             |0           |1140         |0         |0       |
+------------+------------+--------------+-----------+--------------+------------+-------------+----------+--------+



In [79]:
df.select([count(when(col(c).isNull(), 1)).alias("cn_"+c)\
           for c in df.columns]).show()

+------------+------------+--------------+-----------+--------------+------------+-------------+----------+--------+
|cn_InvoiceNo|cn_StockCode|cn_Description|cn_Quantity|cn_InvoiceDate|cn_UnitPrice|cn_CustomerID|cn_Country|cn_index|
+------------+------------+--------------+-----------+--------------+------------+-------------+----------+--------+
|           0|           0|            10|          0|             0|           0|         1140|         0|       0|
+------------+------------+--------------+-----------+--------------+------------+-------------+----------+--------+



In [80]:
df.select(count(when(isnan("Description"), 1))).show()

+----------------------------------------------+
|count(CASE WHEN isnan(Description) THEN 1 END)|
+----------------------------------------------+
|                                             0|
+----------------------------------------------+



In [81]:
df.select(count(when(isnan("Description"), 1).otherwise(0))).show()

+-----------------------------------------------------+
|count(CASE WHEN isnan(Description) THEN 1 ELSE 0 END)|
+-----------------------------------------------------+
|                                                 3108|
+-----------------------------------------------------+



In [82]:
df.select(when(isnan("Description"), 1).otherwise(0).alias("nanCount")).groupBy("nanCount").count().show()

+--------+-----+
|nanCount|count|
+--------+-----+
|       0| 3108|
+--------+-----+



In [83]:
df.select(when(isnan("Description"), 1).otherwise(0)).show(5)

+----------------------------------------------+
|CASE WHEN isnan(Description) THEN 1 ELSE 0 END|
+----------------------------------------------+
|                                             0|
|                                             0|
|                                             0|
|                                             0|
|                                             0|
+----------------------------------------------+
only showing top 5 rows



In [84]:
df.select(when(col("Description").isNull(), 1).otherwise(0).alias("nullCount")).groupBy("nullCount").count().show()

+---------+-----+
|nullCount|count|
+---------+-----+
|        1|   10|
|        0| 3098|
+---------+-----+



In [85]:
print(df.select(col("Description")).count())
print(df.where(col("Description").isNotNull()).count())
print(df.where(col("Description").isNull()).count())
print(df.select(when(isnan("Description"), 1)).count())

print(df.where(col("Description").eqNullSafe("WHITE METAL LANTERN")).count())
print(df.where(col("Description") == "WHITE METAL LANTERN").count())
print(df.where(col("Description") != "WHITE METAL LANTERN").count())

3108
3098
10
3108
6
6
3092


In [86]:
# COMMAND ----------

from pyspark.sql.functions import coalesce
df.select("Description", "CustomerId", coalesce(col("Description"), col("CustomerId"))).show(5, False)
# returns first column that is not null

+-----------------------------------+----------+-----------------------------------+
|Description                        |CustomerId|coalesce(Description, CustomerId)  |
+-----------------------------------+----------+-----------------------------------+
|WHITE HANGING HEART T-LIGHT HOLDER |17850.0   |WHITE HANGING HEART T-LIGHT HOLDER |
|WHITE METAL LANTERN                |17850.0   |WHITE METAL LANTERN                |
|CREAM CUPID HEARTS COAT HANGER     |17850.0   |CREAM CUPID HEARTS COAT HANGER     |
|KNITTED UNION FLAG HOT WATER BOTTLE|17850.0   |KNITTED UNION FLAG HOT WATER BOTTLE|
|RED WOOLLY HOTTIE WHITE HEART.     |17850.0   |RED WOOLLY HOTTIE WHITE HEART.     |
+-----------------------------------+----------+-----------------------------------+
only showing top 5 rows



In [87]:
df.count()
# DEscription nulls 10
#ID  1140

3108

In [88]:
df.select(when(col("Description").isNull(), 1).otherwise(0).alias("nullCount")).groupBy("nullCount").count().show()

+---------+-----+
|nullCount|count|
+---------+-----+
|        1|   10|
|        0| 3098|
+---------+-----+



In [89]:
df.select(when(col("CustomerId").isNull(), 1).otherwise(0).alias("nullCount")).groupBy("nullCount").count().show()

+---------+-----+
|nullCount|count|
+---------+-----+
|        1| 1140|
|        0| 1968|
+---------+-----+



In [90]:
df.select(when(col("Description").isNull() & col("CustomerId").isNull(), 1)\
          .otherwise(0).alias("nullCount")).groupBy("nullCount").count().show()

+---------+-----+
|nullCount|count|
+---------+-----+
|        1|   10|
|        0| 3098|
+---------+-----+



In [91]:
df.select(when(col("Description").isNull() | col("CustomerId").isNull(), 1)\
          .otherwise(0).alias("nullCount")).groupBy("nullCount").count().show()

+---------+-----+
|nullCount|count|
+---------+-----+
|        1| 1140|
|        0| 1968|
+---------+-----+



In [92]:
# COMMAND ----------

na_dropped = df.na.drop("all", subset=["Description", "CustomerId"])
na_dropped.count()

3098

In [93]:
df.na.drop("any", subset=["Description", "CustomerId"]).count()

1968

In [94]:
# COMMAND ----------

da_filled = df.na.fill("any", subset=["Description", "StockCode"])
da_filled.select(when(col("Description").isNull() & col("CustomerId").isNull(), 1)\
          .otherwise(0).alias("nullCount")).groupBy("nullCount").count().show()

+---------+-----+
|nullCount|count|
+---------+-----+
|        0| 3108|
+---------+-----+



In [95]:
# COMMAND ----------

fill_cols_vals = {"StockCode": 5, "Description" : "No Value"}
da_filled = df.na.fill(fill_cols_vals)
da_filled.where("Description = 'No Value'").count()

10

In [96]:
da_filled.where("Description = 'No Value'").show()

+---------+---------+-----------+--------+-------------------+---------+----------+--------------+-----+
|InvoiceNo|StockCode|Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|index|
+---------+---------+-----------+--------+-------------------+---------+----------+--------------+-----+
|   536414|    22139|   No Value|      56|2010-12-01 11:52:00|      0.0|      null|United Kingdom|  622|
|   536545|    21134|   No Value|       1|2010-12-01 14:32:00|      0.0|      null|United Kingdom| 1970|
|   536546|    22145|   No Value|       1|2010-12-01 14:33:00|      0.0|      null|United Kingdom| 1971|
|   536547|    37509|   No Value|       1|2010-12-01 14:33:00|      0.0|      null|United Kingdom| 1972|
|   536549|   85226A|   No Value|       1|2010-12-01 14:34:00|      0.0|      null|United Kingdom| 1987|
|   536550|    85044|   No Value|       1|2010-12-01 14:34:00|      0.0|      null|United Kingdom| 1988|
|   536552|    20950|   No Value|       1|2010-12-01 14

In [97]:
# COMMAND ----------
df_replaced = da_filled.na.replace(["No Value"], ["UNKNOWN"], "Description")

In [98]:
df_replaced.where("Description = 'UNKNOWN'").show()
#without backticks it doest work

+---------+---------+-----------+--------+-------------------+---------+----------+--------------+-----+
|InvoiceNo|StockCode|Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|index|
+---------+---------+-----------+--------+-------------------+---------+----------+--------------+-----+
|   536414|    22139|    UNKNOWN|      56|2010-12-01 11:52:00|      0.0|      null|United Kingdom|  622|
|   536545|    21134|    UNKNOWN|       1|2010-12-01 14:32:00|      0.0|      null|United Kingdom| 1970|
|   536546|    22145|    UNKNOWN|       1|2010-12-01 14:33:00|      0.0|      null|United Kingdom| 1971|
|   536547|    37509|    UNKNOWN|       1|2010-12-01 14:33:00|      0.0|      null|United Kingdom| 1972|
|   536549|   85226A|    UNKNOWN|       1|2010-12-01 14:34:00|      0.0|      null|United Kingdom| 1987|
|   536550|    85044|    UNKNOWN|       1|2010-12-01 14:34:00|      0.0|      null|United Kingdom| 1988|
|   536552|    20950|    UNKNOWN|       1|2010-12-01 14

In [102]:
# COMMAND ----------

from pyspark.sql.functions import struct
complexDF = df.select(struct("Description", "InvoiceNo").alias("complex"))
complexDF.createOrReplaceTempView("complexDF")

In [103]:
complexDF.show(5,False)

+---------------------------------------------+
|complex                                      |
+---------------------------------------------+
|[WHITE HANGING HEART T-LIGHT HOLDER, 536365] |
|[WHITE METAL LANTERN, 536365]                |
|[CREAM CUPID HEARTS COAT HANGER, 536365]     |
|[KNITTED UNION FLAG HOT WATER BOTTLE, 536365]|
|[RED WOOLLY HOTTIE WHITE HEART., 536365]     |
+---------------------------------------------+
only showing top 5 rows



In [105]:
complexDF.select("complex.Description", "complex.InvoiceNo").show(5,False)

+-----------------------------------+---------+
|Description                        |InvoiceNo|
+-----------------------------------+---------+
|WHITE HANGING HEART T-LIGHT HOLDER |536365   |
|WHITE METAL LANTERN                |536365   |
|CREAM CUPID HEARTS COAT HANGER     |536365   |
|KNITTED UNION FLAG HOT WATER BOTTLE|536365   |
|RED WOOLLY HOTTIE WHITE HEART.     |536365   |
+-----------------------------------+---------+
only showing top 5 rows



In [123]:
complexDF.selectExpr("complex.*").show(5,False)

+-----------------------------------+---------+
|Description                        |InvoiceNo|
+-----------------------------------+---------+
|WHITE HANGING HEART T-LIGHT HOLDER |536365   |
|WHITE METAL LANTERN                |536365   |
|CREAM CUPID HEARTS COAT HANGER     |536365   |
|KNITTED UNION FLAG HOT WATER BOTTLE|536365   |
|RED WOOLLY HOTTIE WHITE HEART.     |536365   |
+-----------------------------------+---------+
only showing top 5 rows



In [125]:
complexDF.select(col("complex").getField("Description")).show(5,False)

+-----------------------------------+
|complex.Description                |
+-----------------------------------+
|WHITE HANGING HEART T-LIGHT HOLDER |
|WHITE METAL LANTERN                |
|CREAM CUPID HEARTS COAT HANGER     |
|KNITTED UNION FLAG HOT WATER BOTTLE|
|RED WOOLLY HOTTIE WHITE HEART.     |
+-----------------------------------+
only showing top 5 rows



In [106]:
# COMMAND ----------

from pyspark.sql.functions import split
df.select(split(col("Description"), " ")).show(2, False)

+----------------------------------------+
|split(Description,  )                   |
+----------------------------------------+
|[WHITE, HANGING, HEART, T-LIGHT, HOLDER]|
|[WHITE, METAL, LANTERN]                 |
+----------------------------------------+
only showing top 2 rows



In [118]:
# COMMAND ----------

df.withColumn("array_col", split(col("Description"), " ")).selectExpr("Description","array_col[0]").show(4,False)

+-----------------------------------+------------+
|Description                        |array_col[0]|
+-----------------------------------+------------+
|WHITE HANGING HEART T-LIGHT HOLDER |WHITE       |
|WHITE METAL LANTERN                |WHITE       |
|CREAM CUPID HEARTS COAT HANGER     |CREAM       |
|KNITTED UNION FLAG HOT WATER BOTTLE|KNITTED     |
+-----------------------------------+------------+
only showing top 4 rows



In [121]:
# COMMAND ----------

from pyspark.sql.functions import size
df.select(size(split(col("Description"), " ")), col("Description")).show(5, False) # shows 5 and 3

+---------------------------+-----------------------------------+
|size(split(Description,  ))|Description                        |
+---------------------------+-----------------------------------+
|5                          |WHITE HANGING HEART T-LIGHT HOLDER |
|3                          |WHITE METAL LANTERN                |
|5                          |CREAM CUPID HEARTS COAT HANGER     |
|6                          |KNITTED UNION FLAG HOT WATER BOTTLE|
|5                          |RED WOOLLY HOTTIE WHITE HEART.     |
+---------------------------+-----------------------------------+
only showing top 5 rows



In [122]:
# COMMAND ----------

from pyspark.sql.functions import array_contains
df.select(array_contains(split(col("Description"), " "), "WHITE"), col("Description")).show(5, False)

+--------------------------------------------+-----------------------------------+
|array_contains(split(Description,  ), WHITE)|Description                        |
+--------------------------------------------+-----------------------------------+
|true                                        |WHITE HANGING HEART T-LIGHT HOLDER |
|true                                        |WHITE METAL LANTERN                |
|false                                       |CREAM CUPID HEARTS COAT HANGER     |
|false                                       |KNITTED UNION FLAG HOT WATER BOTTLE|
|true                                        |RED WOOLLY HOTTIE WHITE HEART.     |
+--------------------------------------------+-----------------------------------+
only showing top 5 rows



In [131]:
# COMMAND ----------

from pyspark.sql.functions import split, explode

df.withColumn("splitted", split(col("Description"), " "))\
  .withColumn("exploded", explode(col("splitted")))\
  .select("InvoiceNo","Description", "splitted", "exploded").show(10, False)

+---------+----------------------------------+----------------------------------------+--------+
|InvoiceNo|Description                       |splitted                                |exploded|
+---------+----------------------------------+----------------------------------------+--------+
|536365   |WHITE HANGING HEART T-LIGHT HOLDER|[WHITE, HANGING, HEART, T-LIGHT, HOLDER]|WHITE   |
|536365   |WHITE HANGING HEART T-LIGHT HOLDER|[WHITE, HANGING, HEART, T-LIGHT, HOLDER]|HANGING |
|536365   |WHITE HANGING HEART T-LIGHT HOLDER|[WHITE, HANGING, HEART, T-LIGHT, HOLDER]|HEART   |
|536365   |WHITE HANGING HEART T-LIGHT HOLDER|[WHITE, HANGING, HEART, T-LIGHT, HOLDER]|T-LIGHT |
|536365   |WHITE HANGING HEART T-LIGHT HOLDER|[WHITE, HANGING, HEART, T-LIGHT, HOLDER]|HOLDER  |
|536365   |WHITE METAL LANTERN               |[WHITE, METAL, LANTERN]                 |WHITE   |
|536365   |WHITE METAL LANTERN               |[WHITE, METAL, LANTERN]                 |METAL   |
|536365   |WHITE METAL LANTERN

In [132]:
# COMMAND ----------

from pyspark.sql.functions import create_map
df.select(create_map(col("Description"), col("InvoiceNo")).alias("complex_map"))\
  .show(5,False)

+-----------------------------------------------+
|complex_map                                    |
+-----------------------------------------------+
|[WHITE HANGING HEART T-LIGHT HOLDER -> 536365] |
|[WHITE METAL LANTERN -> 536365]                |
|[CREAM CUPID HEARTS COAT HANGER -> 536365]     |
|[KNITTED UNION FLAG HOT WATER BOTTLE -> 536365]|
|[RED WOOLLY HOTTIE WHITE HEART. -> 536365]     |
+-----------------------------------------------+
only showing top 5 rows



In [135]:
# COMMAND ----------

df.select(create_map(col("Description"), col("InvoiceNo")).alias("complex_map"))\
  .selectExpr("complex_map", "complex_map['WHITE METAL LANTERN']").show(5,False)

+-----------------------------------------------+--------------------------------+
|complex_map                                    |complex_map[WHITE METAL LANTERN]|
+-----------------------------------------------+--------------------------------+
|[WHITE HANGING HEART T-LIGHT HOLDER -> 536365] |null                            |
|[WHITE METAL LANTERN -> 536365]                |536365                          |
|[CREAM CUPID HEARTS COAT HANGER -> 536365]     |null                            |
|[KNITTED UNION FLAG HOT WATER BOTTLE -> 536365]|null                            |
|[RED WOOLLY HOTTIE WHITE HEART. -> 536365]     |null                            |
+-----------------------------------------------+--------------------------------+
only showing top 5 rows



In [140]:
# COMMAND ----------

df.withColumn("complex_map", create_map(col("Description"), col("InvoiceNo")))\
  .selectExpr("InvoiceNo","explode(complex_map)", "complex_map").show(10, False)
    #explode here which will turn them into columns

+---------+-----------------------------------+------+-----------------------------------------------+
|InvoiceNo|key                                |value |complex_map                                    |
+---------+-----------------------------------+------+-----------------------------------------------+
|536365   |WHITE HANGING HEART T-LIGHT HOLDER |536365|[WHITE HANGING HEART T-LIGHT HOLDER -> 536365] |
|536365   |WHITE METAL LANTERN                |536365|[WHITE METAL LANTERN -> 536365]                |
|536365   |CREAM CUPID HEARTS COAT HANGER     |536365|[CREAM CUPID HEARTS COAT HANGER -> 536365]     |
|536365   |KNITTED UNION FLAG HOT WATER BOTTLE|536365|[KNITTED UNION FLAG HOT WATER BOTTLE -> 536365]|
|536365   |RED WOOLLY HOTTIE WHITE HEART.     |536365|[RED WOOLLY HOTTIE WHITE HEART. -> 536365]     |
|536365   |SET 7 BABUSHKA NESTING BOXES       |536365|[SET 7 BABUSHKA NESTING BOXES -> 536365]       |
|536365   |GLASS STAR FROSTED T-LIGHT HOLDER  |536365|[GLASS STAR FROSTED

In [141]:
# COMMAND ----------

jsonDF = spark.range(1).selectExpr("""
  '{"myJSONKey" : {"myJSONValue" : [1, 2, 3]}}' as jsonString""")

In [143]:
jsonDF.show(5, False)

+-------------------------------------------+
|jsonString                                 |
+-------------------------------------------+
|{"myJSONKey" : {"myJSONValue" : [1, 2, 3]}}|
+-------------------------------------------+



In [149]:
# COMMAND ----------

from pyspark.sql.functions import get_json_object, json_tuple

jsonDF.withColumn("get_json_object",get_json_object(col("jsonString"), "$.myJSONKey.myJSONValue[1]"))\
      .withColumn("json_tuple",json_tuple(col("jsonString"), "myJSONKey"))\
      .select("get_json_object","json_tuple").show(2,False)

+---------------+-----------------------+
|get_json_object|json_tuple             |
+---------------+-----------------------+
|2              |{"myJSONValue":[1,2,3]}|
+---------------+-----------------------+



In [151]:
# COMMAND ----------

from pyspark.sql.functions import to_json
df.selectExpr("(InvoiceNo, Description) as myStruct")\
  .withColumn("to_json", to_json(col("myStruct"))).select("myStruct","to_json").show(5,False)

+---------------------------------------------+--------------------------------------------------------------------------+
|myStruct                                     |to_json                                                                   |
+---------------------------------------------+--------------------------------------------------------------------------+
|[536365, WHITE HANGING HEART T-LIGHT HOLDER] |{"InvoiceNo":"536365","Description":"WHITE HANGING HEART T-LIGHT HOLDER"} |
|[536365, WHITE METAL LANTERN]                |{"InvoiceNo":"536365","Description":"WHITE METAL LANTERN"}                |
|[536365, CREAM CUPID HEARTS COAT HANGER]     |{"InvoiceNo":"536365","Description":"CREAM CUPID HEARTS COAT HANGER"}     |
|[536365, KNITTED UNION FLAG HOT WATER BOTTLE]|{"InvoiceNo":"536365","Description":"KNITTED UNION FLAG HOT WATER BOTTLE"}|
|[536365, RED WOOLLY HOTTIE WHITE HEART.]     |{"InvoiceNo":"536365","Description":"RED WOOLLY HOTTIE WHITE HEART."}     |
+---------------

In [158]:
# COMMAND ----------

from pyspark.sql.functions import from_json
from pyspark.sql.types import *

parseSchema = StructType((
  StructField("InvoiceNo",StringType(),True),
  StructField("Description",StringType(),True)))

json = df.selectExpr("(InvoiceNo, Description) as myStruct")\
  .select(to_json(col("myStruct")).alias("to_json"))\
  .select(from_json(col("to_json"), parseSchema).alias("from_json"), col("to_json"))

In [159]:
json.show(5,False)

+---------------------------------------------+--------------------------------------------------------------------------+
|from_json                                    |to_json                                                                   |
+---------------------------------------------+--------------------------------------------------------------------------+
|[536365, WHITE HANGING HEART T-LIGHT HOLDER] |{"InvoiceNo":"536365","Description":"WHITE HANGING HEART T-LIGHT HOLDER"} |
|[536365, WHITE METAL LANTERN]                |{"InvoiceNo":"536365","Description":"WHITE METAL LANTERN"}                |
|[536365, CREAM CUPID HEARTS COAT HANGER]     |{"InvoiceNo":"536365","Description":"CREAM CUPID HEARTS COAT HANGER"}     |
|[536365, KNITTED UNION FLAG HOT WATER BOTTLE]|{"InvoiceNo":"536365","Description":"KNITTED UNION FLAG HOT WATER BOTTLE"}|
|[536365, RED WOOLLY HOTTIE WHITE HEART.]     |{"InvoiceNo":"536365","Description":"RED WOOLLY HOTTIE WHITE HEART."}     |
+---------------

In [160]:
json.printSchema()

root
 |-- from_json: struct (nullable = true)
 |    |-- InvoiceNo: string (nullable = true)
 |    |-- Description: string (nullable = true)
 |-- to_json: string (nullable = true)



In [8]:
# COMMAND ----------

udfExampleDF = spark.range(5).toDF("num")

def power3_1(double_value):
  return float(double_value ** 3)

def power3_2(double_value):
  return double_value ** 3

In [9]:
print(power3_1(2) ,power3_1(2.0), power3_2(2) ,power3_2(2.0))

(8.0, 8.0, 8, 8.0)


In [10]:
# COMMAND ----------

from pyspark.sql.functions import udf
power3udf_1 = udf(power3_1)

In [11]:
# COMMAND ----------

from pyspark.sql.functions import col
udfExampleDF.select(power3udf_1(col("num"))).show(5)

+-------------+
|power3_1(num)|
+-------------+
|          0.0|
|          1.0|
|          8.0|
|         27.0|
|         64.0|
+-------------+



In [14]:
# COMMAND ----------

from pyspark.sql.types import IntegerType, DoubleType

#spark.udf.register("power3py_ur_1", power3udf, DoubleType()) #This will not work

#TypeError: Invalid returnType: data type can not be specified when f isa user-defined function, but got DoubleType.
spark.udf.register("power3udf_ur_1", power3udf_1)
udfExampleDF.selectExpr("power3udf_ur_1(num)").show(5)

+-------------------+
|power3udf_ur_1(num)|
+-------------------+
|                0.0|
|                1.0|
|                8.0|
|               27.0|
|               64.0|
+-------------------+



In [15]:
spark.udf.register("power3_1_double", power3_1, DoubleType())
udfExampleDF.selectExpr("power3_1_double(num)").show(5)

+--------------------+
|power3_1_double(num)|
+--------------------+
|                 0.0|
|                 1.0|
|                 8.0|
|                27.0|
|                64.0|
+--------------------+



In [16]:
spark.udf.register("power3_1_int", power3_1, IntegerType())
udfExampleDF.selectExpr("power3_1_int(num)").show(5)

+-----------------+
|power3_1_int(num)|
+-----------------+
|             null|
|             null|
|             null|
|             null|
|             null|
+-----------------+



In [17]:
spark.udf.register("power3_2_double", power3_2, DoubleType())
udfExampleDF.selectExpr("power3_2_double(num)").show(5)

+--------------------+
|power3_2_double(num)|
+--------------------+
|                null|
|                null|
|                null|
|                null|
|                null|
+--------------------+



In [19]:
spark.udf.register("power3_2_int", power3_2, IntegerType())
udfExampleDF.selectExpr("power3_2_int(num)").show(5)

+-----------------+
|power3_2_int(num)|
+-----------------+
|                0|
|                1|
|                8|
|               27|
|               64|
+-----------------+

